In [8]:
import cv2, os
import numpy as np

In [9]:
img_path = 'data/fr.jpg'
file_name, ext = os.path.splitext(os.path.basename(img_path))    # 파일이름, 확장자
org_img = cv2.imread(img_path)   # 원본 이미지 

In [10]:
src = []     # '왼쪽 위', '오른쪽 위', '오른쪽 아래', '왼쪽 아래' 순서 ( 시계방향 ) 로 각 점의 좌표를 넣는다.

In [11]:
# mouse Callback Handler
def mouse_handler(event, x, y, flags, param):   # 이벤트, 마우스의 x좌표, 마우스의 y좌표
    if event == cv2.EVENT_LBUTTONUP:            # 마우스 왼쪽버튼을 눌렀다 똈을때만 행동을 한다.
        img = org_img.copy()
    
        src.append([x, y])    # 마우스의 x,y 좌표 저장 
        
        # 클릭했을때, 그 지점을 '초록색 원'으로 표시
        for xx, yy in src:
            cv2.circle(img, center=(xx, yy), radius=5, color=(0,255,0), thickness=-1, lineType=cv2.LINE_AA)   # cv2.circle(): 원을 그린다.
        
        cv2.imshow('img', img)
        
        # perspective transform : 입력(src)으로 4개, 출력(dest)으로 4개의 점이 필요. (4개의 점을 각각 매칭 시켜줌)
        if len(src) == 4:     # src에 들어있는 점의 개수가 4개일때만
            src_np = np.array(src, dtype=np.float32)    # src에 들어있는점들을 전부 float형으로 변환 
            
            # 만들어낸 이미지의 가로, 세로 정하기
            # 위, 아래 중 긴것을 width로, 왼, 오 중 긴것을 height로 지정 하겠다. (max)
            # numpy.linalg.norm(a - b) : a점과 b점 사이의 거리를 구한다.
            width = max(np.linalg.norm(src_np[0] - src_np[1]), np.linalg.norm(src_np[2] -src_np[3]))
            height = max(np.linalg.norm(src_np[0] - src_np[3]), np.linalg.norm(src_np[1] -src_np[2]))
            
            # 우리가 만들어낼 이미지(dest)의 좌표 정하기
            dest_np = np.array([
                [0, 0],          # 왼쪽 위
                [width, 0],      # 오른쪽 위
                [width, height], # 오른쪽 아래
                [0, height]      # 왼쪽 아래 
            ], dtype=np.float32)
            
            # cv2.getPerspectiveTransform() : PerspectiveTransform 매트릭스를 구한다. (결과로 나올 형태를 지정해줌)
            Matr = cv2.getPerspectiveTransform(src=src_np, dst=dest_np)    
            # cv2.warpPerspective() : 이미지를 변형
            result = cv2.warpPerspective(org_img, M=Matr, dsize=(width, height))   # org_img를 Matr형태로 변환해라. (dsize = 사이즈)
            
            cv2.imshow('result', result)
            cv2.imwrite('data/result/%s_result%s' % (file_name, ext), result)

In [12]:
# cv2.circle?

In [13]:
# main
cv2.namedWindow('img')                         # 윈도우에 이름지정
cv2.setMouseCallback('img', mouse_handler)     # 특정 윈도우에 마우스 콜백을 지정 

cv2.imshow('img', org_img)                     # 이미지 띄우기
cv2.waitKey(0)

-1